In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('wiki_movie_plots_deduped.csv')  

In [3]:
data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [24]:
import regex as re

storydata=data[data["Genre"].str.contains('science fiction|sci-fi|science-fiction|horror|thriller|suspense|drama|comedy|action|adventure|crime|fantasy|mystery|romance|superhero')]
len(storydata)

24169

In [29]:
with open('wikistories.txt', 'w',encoding='utf-8') as f:
    for ind,col in storydata.iterrows():
        f.write("<|startoftext|> ")
        f.write("Genre: "+col['Genre']+" Title: "+col['Title']+" Plot: "+col['Plot']+"<|endoftext|>\n\n")

In [10]:
#scifidata["Plot"]=scifidata["Plot"].apply(cleanText)
with open('wikiscifi.txt', 'w',encoding='utf-8') as f:
    f.write("<|startoftext|> ")
    for rec_index, rec in scifidata.iterrows():
        f.write(rec['Plot'])
    f.write(" <|endoftext|>")    

In [11]:
#horrordata["Plot"]=horrordata["Plot"].apply(cleanText)
with open('wikihorror.txt', 'w',encoding='utf-8') as f:
    f.write("<|startoftext|> ")
    for rec_index, rec in horrordata.iterrows():
        f.write(rec['Plot'])
    f.write(" <|endoftext|>")    

In [12]:
#dramadata["Plot"]=dramadata["Plot"].apply(cleanText)
with open('wikidrama.txt', 'w',encoding='utf-8') as f:
    f.write("<|startoftext|> ")
    for rec_index, rec in dramadata.iterrows():
        f.write(rec['Plot'])
    f.write(" <|endoftext|>")    

In [22]:
def cleanText(x):
    cleanedtext=re.sub("[\(\[].*?[\)\]]", "", x)
    #cleanedtext+="<|endoftext|>"
    cleanedtext=x.replace("\n","")
    return cleanedtext

In [23]:
data["Plot"]=data["Plot"].apply(cleanText)
#unsuffixeddata=pd.DataFrame("<"+data["Genre"]+">"+data["Plot"])

In [ ]:
if "horror" in data["Genre"]:
    print(data['Plot'])
else :
    print("no")

In [ ]:
unsuffixeddata.to_csv('data2.csv', index=False,) 

In [ ]:
textdata.to_csv('data1.csv', index=False,) 

#### Training

In [ ]:
import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

In [ ]:
base_model = GPT2LMHeadModel.from_pretrained('gpt2')
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
bos = '<BOS>'
eos = '<|endoftext|>'
pad = '<|pad|>'
special_tokens_dict = {'eos_token': eos, 'bos_token': bos,'pad_token': pad}

num_added_toks = base_tokenizer.add_special_tokens(special_tokens_dict)

config = AutoConfig.from_pretrained('gpt2', 
                                    bos_token_id=base_tokenizer.bos_token_id,
                                    eos_token_id=base_tokenizer.eos_token_id,
                                    pad_token_id=base_tokenizer.pad_token_id,
                                    output_hidden_states=False)

base_model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)
base_model.resize_token_embeddings(len(base_tokenizer))

In [ ]:
df_train, df_val = train_test_split(textdata, train_size = 0.9, random_state = 77)
print(f'There are {len(df_train)} stories for training and {len(df_val)} for validation')

In [ ]:
pip install datasets

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train[['Plot']])
val_dataset = Dataset.from_pandas(df_val[['Plot']])

In [ ]:
 def tokenize_function(examples,tokenizer):
        return base_tokenizer(examples['Plot'], padding=True)

In [ ]:
tokenized_train_dataset = train_dataset.map(
    tokenize_function(train_dataset,tokenizer=base_tokenizer),
    batched=True,
    num_proc=5,
)



In [ ]:
tokenized_val_dataset = val_dataset.map(
    tokenize_function(val_dataset,tokenizer=base_tokenizer),
    batched=True,
    num_proc=5,
    verbose=True
)

# Training GPT2SIMPLE

In [ ]:
import gpt_2_simple as gpt2
import os
import requests

model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/


file_name = "data2.csv"
if not os.path.isfile(file_name):
	#url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
	#data = requests.get(url)

	with open(file_name, 'w',encoding='utf-8') as f:
		f.write(data.text)

In [ ]:
import gpt_2_simple as gpt2
model_name = "124M"
file_name = "data2.csv"
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              multi_gpu=True,
              save_every=10,
              sample_length=400,
              steps=1500)

# Old School training

In [1]:
import gpt_2_simple as gpt2

In [ ]:
gpt2.download_gpt2(model_name='124M')

In [2]:
sess = gpt2.start_tf_sess()

In [ ]:
gpt2.finetune(sess, dataset="wikihorror.txt", steps=1000, model_name='124M',
sample_every=200, save_every=10, print_every=1, restore_from='fresh',run_name='horror')#set overwrite false and  restore_from="latest" if crash

In [ ]:
gpt2.finetune(sess, dataset="wikidrama.txt", steps=1000, model_name='124M',
sample_every=200, save_every=10, print_every=1, restore_from='fresh',run_name='drama')

In [ ]:
gpt2.finetune(sess, dataset="wikiscifi.txt", steps=1000, model_name='124M',
sample_every=200, save_every=10, print_every=1, restore_from='fresh',run_name='scifi')

In [3]:
gpt2.load_gpt2(sess, run_name='horror')
print(str(gpt2.generate(sess, run_name='horror', length=450,temperature=0.7, prefix="Jason fell in love with Ginny, what happens?",return_as_list=True)[0]))

Loading checkpoint checkpoint\horror\model-901
INFO:tensorflow:Restoring parameters from checkpoint\horror\model-901
Jason fell in love with Ginny, what happens? And what about her mother? The answer is that she dies of her gunshot wound.

Ginny's mother, Betty Watson, is a voracious reader of the local newspapers and a former lover of the family. She is part of a group of disturbed teenagers who have been following her daughter closely, and she is now in search of answers to the questions she has been asking this whole time.

She starts reading the newspaper, and sees that the newspaper advertises a "religion class". No one is on campus, and she spends a lot of time reading about the story. She also gets into a fight with a classmate, which results in her being gunned down by the police. The teen who ran the fight is the same girl who is the target of the previous murder, and the girl is the main suspect.

Eventually it all comes down to the question of which girl is the real target. 